# WeightWatcher PyTorch Demo

In [ ]:
import numpy as np
import torch
import torchvision.models
import torch.nn as nn

import ww

In [ ]:
%load_ext autoreload
%autoreload 2

## VGG Architecture

In [ ]:
# VGG16 with BatchNorm
model = torchvision.models.vgg16_bn()

In [ ]:
# Draw graph
dg = ww.build_pytorch_graph(model, torch.zeros([1, 3, 224, 224]))
dot = ww.draw_graph(dg)
dot

## ResNet Architecture

In [ ]:
# Resnet50
model = torchvision.models.resnet50()

In [ ]:
# Draw graph
dg = ww.build_pytorch_graph(model, torch.zeros([1, 3, 224, 224]))
ww.draw_graph(dg)